# Транслитерация названий городов для Карьерного центра Яндекс Практикум


### Цель:
- Сопоставление произвольных гео названий с унифицированными именами geonames для внутреннего использования Карьерным центром


### Задачи:


- Создать решение для подбора наиболее подходящих названий с geonames. Например Ереван -> Yerevan


- На примере РФ и стран наиболее популярных для релокации - Беларусь, Армения, Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек (с возможностью масштабирования на сервере заказчика)


- Возвращаемые поля geonameid, name, region, country, cosine similarity
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}] где словарь - одна запись с указанными полями


### Описание данных
Используемые таблицы с geonames:

- admin1CodesASCII
- alternateNamesV2
- cities15000
- countryInfo
- при необходимости любые другие открытые данные




### План:

- Ознакомление с данными;
- Подготовка матрицы с помощью Countvectorizer;
- Подготовка преобразования поступающих данных:
    - транслитерация 1
    - транслитерация 2
    - перевод
- Функция подсчета расстояния между вектрами и вывод топ 3-5 подходящих
- Вывод


## Формат выдачи:
- Возвращаемые поля geonameid, name, region, country, cosine similarity
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}] где словарь - одна запись с указанными полями



`* http://download.geonames.org/export/dump/`\
`** RU, BY, KG, KZ, AM, GE, RS, ME`

# Загрузка библиотек

In [237]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

!pip install Unidecode
from unidecode import unidecode

!pip install transliterate
from transliterate import translit

# Загрузка данных

In [238]:
# names in English for admin divisions. Columns: code, name, name ascii, geonameid
adm_codes = pd.read_csv(
    '/content/drive/MyDrive/translit/data/admin1CodesASCII.txt',
    names=['code', 'name', 'region', 'geonameid'],
    usecols=['code', 'region'],
    header=None,
    sep='\t',
    on_bad_lines='skip'
    )

# # alternate names with language codes and geonameId, file with iso language codes, with new columns from and to
# alt_names = pd.read_csv(
#     '/content/drive/MyDrive/translit/data/alternateNamesV2.txt',
#     names=[
#         'alternateNameId', 'geonameid', 'isolanguage', 'alternate_name',
#         'isPreferredName', 'isShortName', 'isColloquial', 'isHistoric',
#         'from', 'to'
#         ],
#     sep='\t',
#     on_bad_lines='skip'
#     )

# alt_names['isolanguage_upper'] = alt_names['isolanguage'].str.upper()
# alt_names = alt_names.query('isolanguage_upper in @task_country_codes')


# all cities with a population > 15000 or capitals (ca 25.000), see 'geoname' table for columns
cities = pd.read_csv(
    '/content/drive/MyDrive/translit/data/cities15000.txt',
    names=[
        'geonameid', 'city', 'name', 'alternatenames',
        'latitude', 'longitude', 'feature_class', 'feature_code',
        'country_code', 'cc2', 'admin1_code', 'admin2_code',
        'admin3_code', 'admin4_code', 'population', 'elevation',
        'dem', 'timezone', 'modification_date'
        ],
    usecols=[
        'geonameid', 'name',
        'alternatenames', 'country_code', 'admin1_code'
        ],
    sep='\t',
    on_bad_lines='skip'
    )

# country information : iso codes, fips codes, languages, capital ,...
country = pd.read_csv(
    '/content/drive/MyDrive/translit/data/countryInfo.txt',
    names=[
        'ISO', 'ISO3', 'ISO_Numeric', 'fips',
        'country', 'Capital_Area_sq_km', 'Population_Continent', 'tld',
        'CurrencyCode', 'CurrencyName', 'Phone', 'Postal_Code',
        'Format', 'Postal_Code2', 'Regex', 'Languages',
        'geonameid', 'neighbours', 'EquivalentFipsCode'
    ],
    usecols=['ISO', 'country'],
    sep='\t', skiprows=50,
    on_bad_lines='skip')

In [239]:
task_country_codes = ['RU', 'BY', 'KG', 'KZ', 'AM', 'GE', 'RS', 'ME']

### adm_codes

In [240]:
adm_codes[:3]

,code,region
0,AD.06,Sant Julia de Loria
1,AD.05,Ordino
2,AD.04,La Massana


In [241]:
adm_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3881 entries, 0 to 3880
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    3881 non-null   object
 1   region  3881 non-null   object
dtypes: object(2)
memory usage: 60.8+ KB


In [242]:
adm_codes['country_code'] = adm_codes['code'].apply(lambda x: x.split('.')[0])
adm_codes['admin1_code'] = adm_codes['code'].apply(lambda x: x.split('.')[1])

adm_codes = adm_codes.drop('code', axis=1)

In [243]:
adm_codes = adm_codes.query('country_code in @task_country_codes')

In [244]:
adm_codes

,region,country_code,admin1_code
82,Ararat,AM,02
83,Syunik,AM,08
84,Vayots Dzor,AM,10
85,Yerevan,AM,11
86,Aragatsotn,AM,01
...,...,...,...
2835,Sakhalin Oblast,RU,64
2836,Magadan Oblast,RU,44
2837,Kamchatka,RU,92
2838,Chukotka,RU,15


### cities

In [245]:
cities = cities.query('country_code in @task_country_codes')

In [246]:
cities

,geonameid,name,alternatenames,country_code,admin1_code
94,174875,Kapan,"Ghap'an,Ghapan,Ghap’an,Kafan,Kafin,Kapan,Kapan...",AM,08
95,174895,Goris,"Geryusy,Goris,Горис,Գորիս",AM,08
96,174972,Hats'avan,"Acavan,Atsavan,Hats'avan,Hats’avan,Sisian,Ацав...",AM,08
97,174979,Artashat,"Artachat,Artasat,Artasatas,Artasato,Artaschat,...",AM,02
98,174991,Ararat,"Ararat,Araratas,Ararato,Davalinskiy Tsemzavod,...",AM,02
...,...,...,...,...,...
21255,8505053,Vostochnoe Degunino,"Vostochnoe Degunino,Восточное Дегунино",RU,48
21256,8521440,Dzerzhinsky,"Dzerzhinskij,Дзержинский",RU,47
21257,11238229,Obruchevo,NaN,RU,48
21258,11886891,Fedorovskiy,"Fedorovskij,Федоровский",RU,32


In [247]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1348 entries, 94 to 21259
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   geonameid       1348 non-null   int64 
 1   name            1348 non-null   object
 2   alternatenames  1345 non-null   object
 3   country_code    1348 non-null   object
 4   admin1_code     1348 non-null   object
dtypes: int64(1), object(4)
memory usage: 63.2+ KB


### country

In [248]:
country[:3]

,ISO,country
0,AD,Andorra
1,AE,United Arab Emirates
2,AF,Afghanistan


In [249]:
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ISO      251 non-null    object
 1   country  252 non-null    object
dtypes: object(2)
memory usage: 4.1+ KB


## Соберем всю информацию в одну таблицу

In [250]:
df = cities.merge(country, how='inner', left_on='country_code', right_on='ISO')

In [251]:
df = pd.merge(df, adm_codes,  how='left', left_on=['country_code', 'admin1_code'], right_on=['country_code', 'admin1_code'])

In [252]:
df[:3]

,geonameid,name,alternatenames,country_code,admin1_code,ISO,country,region
0,174875,Kapan,"Ghap'an,Ghapan,Ghap’an,Kafan,Kafin,Kapan,Kapan...",AM,08,AM,Armenia,Syunik
1,174895,Goris,"Geryusy,Goris,Горис,Գորիս",AM,08,AM,Armenia,Syunik
2,174972,Hats'avan,"Acavan,Atsavan,Hats'avan,Hats’avan,Sisian,Ацав...",AM,08,AM,Armenia,Syunik


Не во всех `alternatenames` встречаются названия из `name`. Добавим все в один столбец и расширим датасет по получившемуся столбцу

In [253]:
df['all_names'] = df.dropna(subset=['alternatenames']).apply(lambda row: (row['name']+','+row['alternatenames']), axis=1)
df = df.drop('alternatenames', axis=1)

# сформируем список по которому будем расширять датасет
df['all_names'] = df['all_names'].dropna().apply(lambda x: x.split(','))

df = df.explode('all_names').reset_index(drop=True)

print(f'Дубликатов до:{df.duplicated().sum()}')

# иногда значения встречались, дубликаты удалим
df = df.drop_duplicates().reset_index(drop=True)

print(f'Дубликатов после:{df.duplicated().sum()}')

Дубликатов до:1307
Дубликатов после:0


In [254]:
df[3333:3336]

,geonameid,name,country_code,admin1_code,ISO,country,region,all_names
3333,3193044,Podgorica,ME,16,ME,Montenegro,Podgorica,بودغوريتسا
3334,3193044,Podgorica,ME,16,ME,Montenegro,Podgorica,پودگوریتسا
3335,3193044,Podgorica,ME,16,ME,Montenegro,Podgorica,پوڈگوریکا


Удалим все не `ascii` названия

In [255]:
df['ascii'] = df['all_names'].dropna().apply(lambda x: x.isascii())

df = df.query('ascii == True')
df = df.drop('ascii', axis=1).reset_index(drop=True)

Данные подготовлены под работу с CountVectorizer

## Преобразование в вектор

In [256]:
# Создаем объект CountVectorizer
vectorizer = CountVectorizer(
    analyzer='char',
    ngram_range=(1,4),
    # max_df=0.9,
    # min_df=0.1,
    # binary=True,
    )

In [257]:
# Преобразуем названия городов в векторное представление
X = vectorizer.fit_transform(df['all_names'])

In [258]:
y = vectorizer.transform(['Belgorod'])

In [259]:
# Вычисляем матрицу косинусных расстояний
cosine_similarities = cosine_similarity(X, y)

In [260]:
n = 4

x = cosine_similarities.flatten()
max_indices = np.argpartition(x,-n)[-n:]
max_values = x[max_indices]
sorted_max_indices = max_indices[np.argsort(max_values)[::-1]]
sorted_max_values = x[sorted_max_indices]
print('indices of max values:', max_indices)
print('max values:', max_values)
print('sorted indices of max values:', sorted_max_indices)
print('sorted max values:', sorted_max_values)

indices of max values: [8690 8679 8676 8681]
max values: [0.93541435 0.93541435 1.         0.94491118]
sorted indices of max values: [8676 8681 8679 8690]
sorted max values: [1.         0.94491118 0.93541435 0.93541435]


In [261]:
df.loc[sorted_max_indices]

,geonameid,name,country_code,admin1_code,ISO,country,region,all_names
8676,578072,Belgorod,RU,09,RU,Russia,Belgorod Oblast,Belgorod
8681,578072,Belgorod,RU,09,RU,Russia,Belgorod Oblast,Belgorodo
8679,578072,Belgorod,RU,09,RU,Russia,Belgorod Oblast,Belgoroda
8690,578072,Belgorod,RU,09,RU,Russia,Belgorod Oblast,belgoroda


In [262]:
df.loc[sorted_max_indices].to_dict('records')[0]['name']

'Belgorod'

Добавим транслитерацию и подготовим функцию получения готовго названия

In [263]:
# Функция преобразования

def return_dict(city):
    '''
    Возвращаемые поля geonameid, name, region, country, cosine similarity
    '''
    city = unidecode(city)
    y = vectorizer.transform([city])
    cosine_similarities = cosine_similarity(X, y)

    x = cosine_similarities.flatten()
    max_indices = np.argpartition(x,-n)[-n:]
    max_values = x[max_indices]
    sorted_max_indices = max_indices[np.argsort(max_values)[::-1]]
    result = df.drop(['all_names', 'admin1_code', 'ISO', 'country_code']
                     , axis=1).loc[sorted_max_indices].to_dict('records')[0]
    result['cosine_similarity'] = max_values[0]
    return result

Проверим поиск для города Белгород, намеренно совершив опечатку

In [264]:
return_dict('Белгарод')

{'geonameid': 578072,
 'name': 'Belgorod',
 'country': 'Russia',
 'region': 'Belgorod Oblast',
 'cosine_similarity': 0.6240377207533827}

Город корректно найден

# Test

Проверим на тестовом датасете

In [265]:
def return_name(city):
    '''
    Выдача только Названия города для тестирования
    '''
    # city = unidecode(city)
    city = translit(city, reversed=True)
    y = vectorizer.transform([city])
    cosine_similarities = cosine_similarity(X, y)
    n = 1

    x = cosine_similarities.flatten()
    max_indices = np.argpartition(x,-n)[-n:]
    max_values = x[max_indices]
    sorted_max_indices = max_indices[np.argsort(max_values)[::-1]]
    return df.loc[sorted_max_indices].to_dict('records')[0]['name']

In [266]:
test = pd.read_csv('/content/drive/MyDrive/translit/data/geo_test.csv', delimiter=';')

In [267]:
test[:3]

,query,name,region,country
0,Смоленск,Smolensk,Smolensk Oblast,Russia
1,Кемерово,Kemerovo,Kuzbass,Russia
2,Бишкек,Bishkek,Bishkek,Kyrgyzstan


In [268]:
test['name_pred'] = test['query'].apply(return_name)

In [269]:
test[test['name_pred'] != test['name']].sample(10)

,query,name,region,country,name_pred
15,Екб,Yekaterinburg,Sverdlovsk Oblast,Russia,Ekibastuz
216,Чистополь,Chistopol’,Tatarstan Republic,Russia,Chistopol'
166,Новая Адыгея,Novaya Adygeya,Adygeya Republic,Russia,Novaya Derevnya
276,Артем,Artëm,Primorye,Russia,Artem
10,Минск,Minsk City,Minsk City,Belarus,Minsk
14,Рязань,Ryazan’,Ryazan Oblast,Russia,Ryazan'
319,Кокошкино,Kokoshkino,Moscow,Russia,Korkino
17,Н.Новгород,Nizhniy Novgorod,Nizhny Novgorod Oblast,Russia,Velikiy Novgorod
175,Щекино,Shchëkino,Tula Oblast,Russia,Shchekino
119,Джанкой,Zhanibek,Batys Qazaqstan,Kazakhstan,Zhangatas


In [270]:
print(f"Доля верно распознанных {sum(test['name_pred'] == test['name'])/len(test):0.3f}")

Доля верно распознанных 0.861


# Вывод:

По результатам данной работы было разработано решение по преобразованию подаваемого на вход названия населенного пункта из заданного списка стран, с выводом информации о стране, имени, региону, id, и уверенности предсказания в виде косинусного расстояния между найденным значением и подданым на вход.

Для поиска и преобразования данных были использованы несколько таблиц с сайта `geonames.org`. Они были преобразованны: удалена не требующаяся информация, соеденены несколько таблиц в одну.

Для построения матрицы признаков был использован CountVectorizer со следующими параметрами:

В результате проверки на тестовом датасете точность работы составила 86% верно распознанных названий.

Методы дальнейшего повышения качества: добавление алтернативных названий из дополнительных таблиц, обработка некоторых названий с заменой по словарю.